**input_vars**
* sent latnecy inflation: (rtt_samples[second_half]-rtt_samples[first_half])/send_duration
* latency ratio: mean(rtt_samples)/min_latency
* recv ratio: recv_rate/send_rate
---
* min_first_var, max_first_var: -0.49 0.38
* min_second_var, max_second_var: 1.0 3.54
* min_third_var, max_third_var: 0.08 9.04
* min_action, max_action: -6.25 9.02
---
* min_first_var, max_first_var: -0.46, 0.76
* min_second_var, max_second_var: 1.0 43.64
* min_third_var, max_third_var: 0.0, 9.75
* min_action, max_action: -68.32 41.78

* if inflation is lower, the rate should not be higher
* if latency is lower, the rate should not be higher
* if recv ratio is lower, 

In [2]:
input_range_dict = {
    1: [(-0.1, 0.1), (1.5, 2.5), (0.4, 0.7)],
    2: [(-0.3, 0.3), (1.5, 3.5), (0.2, 0.8)],
    3: [(-0.5, 0.5), (1, 4), (0.0, 1.0)],
}

# Proporty Definition

## Robustness

In [3]:
DECLARATION_LINE_BASE = "(declare-const {} Real)"
INPUT_CONSTRAINT_LINE_BASE = "(assert ({} {} {}))"
OUTPUT_CONSTRAINT_LINE_BASE = "(assert (<= {} {}))"

if_multistep = False

def gen_prop(difficulty):
    lines = []
    lines.append("; variable declaration")
    for i in range(2*model_input_size):
        lines.append(DECLARATION_LINE_BASE.format(f"X_{i}"))

    if if_differential:
        lines.append(DECLARATION_LINE_BASE.format("Y_0"))
    else:
        lines.append(DECLARATION_LINE_BASE.format("Y_0"))
        lines.append(DECLARATION_LINE_BASE.format("Y_1"))

    lines.append(" ")
    lines.append("; input constraints")
    for i in range(model_input_size):
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{i}", input_range_dict[difficulty][int(i%history_length)][0]))
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{i}", input_range_dict[difficulty][int(i%history_length)][1]))

    for i in range(model_input_size):
        if if_multistep or int(i/history_length) == history_length-1:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{model_input_size+i}", -epsilon))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", epsilon))
        else:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{model_input_size+i}", 0))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", 0))

    lines.append(" ")
    lines.append("; output constraints")

    if if_differential:
        if if_a_less_b:
            lines.append("(assert (<= Y_0 {}))".format(-tolerance))
        else:
            lines.append("(assert (>= Y_0 {}))".format(tolerance))
    else:
        if if_a_less_b:
            lines.append("(assert (<= Y_0 {}))".format(-tolerance/2))
            lines.append("(assert (>= Y_1 {}))".format(tolerance/2))
        else:
            lines.append("(assert (>= Y_0 {}))".format(tolerance/2))
            lines.append("(assert (<= Y_1 {}))".format(-tolerance/2))

    path = base_path.format(difficulty, str(epsilon)[2:], "a_less_b" if if_a_less_b else "b_less_a",
        "multistep" if if_multistep else "singlestep", "_differential" if if_differential else "")

    f = open(path, "w")
    f.write('\n'.join(lines))
    f.close()

In [4]:
model_input_size = 9
history_length = 3
epsilon = 0.001
if_multistep = True
if_differential = True
if_a_less_b = False
tolerance = 0.5

base_path = "vnnlib/robustness/difficulty{}_eps{}_{}_{}{}.vnnlib"

for if_multistep in [True, False]:
    for if_differential in [True, False]:
        for if_a_less_b in [True, False]:
            for difficulty in [1, 2, 3]:
                gen_prop(difficulty=difficulty)

## Monotonicity

In [4]:
DECLARATION_LINE_BASE = "(declare-const {} Real)"
INPUT_CONSTRAINT_LINE_BASE = "(assert ({} {} {}))"
OUTPUT_CONSTRAINT_LINE_BASE = "(assert (<= {} {}))"

def gen_prop(difficulty, feat_index):
    lines = []
    lines.append("; variable declaration")
    for i in range(2*model_input_size):
        lines.append(DECLARATION_LINE_BASE.format(f"X_{i}"))

    if if_differential:
        lines.append(DECLARATION_LINE_BASE.format("Y_0"))
    else:
        lines.append(DECLARATION_LINE_BASE.format("Y_0"))
        lines.append(DECLARATION_LINE_BASE.format("Y_1"))

    lines.append(" ")
    lines.append("; input constraints")
    for i in range(model_input_size):
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{i}", input_range_dict[difficulty][int(i%history_length)][0]))
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{i}", input_range_dict[difficulty][int(i%history_length)][1]))

    for i in range(model_input_size):
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{model_input_size+i}", 0))
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", 0))

        if int(i % history_length) in feat_index:
            if if_multistep or (int(i/history_length) == history_length-1):
                lines[-1] = INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", epsilon)

    lines.append(" ")
    lines.append("; output constraints")
    if if_differential:
        if if_a_less_b:
            lines.append("(assert (<= Y_0 {}))".format(-tolerance))
        else:
            lines.append("(assert (>= Y_0 {}))".format(tolerance))
    else:
        if if_a_less_b:
            lines.append("(assert (<= Y_0 {}))".format(-tolerance/2))
            lines.append("(assert (>= Y_1 {}))".format(tolerance/2))
        else:
            lines.append("(assert (>= Y_0 {}))".format(tolerance/2))
            lines.append("(assert (<= Y_1 {}))".format(-tolerance/2))

    path = base_path.format(difficulty, str(epsilon)[2:])

    f = open(path, "w")
    f.write('\n'.join(lines) )
    f.close()
    

In [6]:
model_input_size = 9
history_length = 3
epsilon = 0.01
tolerance = 0.5

# recv_ratio_idx = [2]
# if_a_less_b = False
# base_path = "vnnlib/loss_avoidance/difficulty{}_eps{}_singlestep.vnnlib"

latency_ratio_idx = [1]
if_a_less_b = True
base_path = "vnnlib/capacity_utilization/difficulty{}_eps{}_singlestep.vnnlib"

if_multistep = False
if_differential = False

for difficulty in [1, 2, 3]:
    for epsilon in [0.01, 0.001]:
        gen_prop(difficulty=difficulty, feat_index=latency_ratio_idx)
